In [1]:
import os, sys
path_to_package = os.path.abspath(os.path.join('../'))
if path_to_package not in sys.path:
    sys.path.append(path_to_package)

import matplotlib.pyplot as plt
import pandas as pd
import json
import glob

from src.utils import check_working_os
from src.io import get_filepaths, write_json

In [2]:
def load_index(filename):
    index_path = "/home/khaldrem/code/sc_regmod/dataset/index"
    f = open(f"{index_path}/{filename}.json", "r")
    json_data = json.load(f)
    f.close()

    return json_data
    

In [3]:
phenotypes = [
    "SM300-Efficiency", "SM300-Rate", "SM300-Lag", "SM300-AUC",
    "SM60-Efficiency",  "SM60-Rate",  "SM60-Lag",  "SM60-AUC",
    "Ratio-Efficiency", "Ratio-Rate", "Ratio-Lag", "Ratio-AUC",
]

exp = ["1", "2", "3", "4"]
chromosomes = ["all", "haploide-euploide", "diploides-euploides", "---"]
anova_path = "/home/khaldrem/code/sc_regmod/dataset/anova/anova_at_least_one_phenotype/p_value_0_01/"

In [4]:
#Limpiar ANOVA en el index
for file in glob.glob("/home/khaldrem/code/sc_regmod/dataset/index/*.json"):
    f = open(file, "r")
    json_data = json.load(f)
    f.close()

    if 'anova' in json_data.keys():
        json_data["anova"] = {}


In [9]:
#Por cada archivo en el ANOVA
for exp, chromosome in zip(exp, chromosomes):
    print(f"exp: {exp} | chrosomose: {chromosome}")

    filepaths = glob.glob(f"/home/khaldrem/code/sc_regmod/dataset/anova/anova_at_least_one_phenotype/p_value_0_01/{chromosome}/csv/*.csv")
    for fp in filepaths:
        filename = fp.split("/")[-1].split(".")[0]
        data_csv = pd.read_csv(fp)

        unique_pos = set()

        for idx, row in data_csv.iterrows():
            for pt in phenotypes:
                if row[f"{pt}_pvalue"] < 0.01:
                    unique_pos.add(int(row["col"]))

        # print(f"filename: {filename} | len: {len(unique_pos)} pos: {unique_pos}")

        index = load_index(filename)
        index['anova'][exp] = sorted(list(unique_pos))

        with open(f"/home/khaldrem/code/sc_regmod/dataset/index/{filename}.json", 'w') as outfile:
            json.dump(index, outfile)
    

exp: 4 | chrosomose: all


In [6]:
yil = load_index("YJL123C")

In [8]:
len(yil["anova"]["1"])

137